In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/konwinski-prize/data.a_zip
/kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.1-py3-none-any.whl
/kaggle/input/konwinski-prize/kprize_setup/uv/uv
/kaggle/input/konwinski-prize/kprize_setup/uv/uvx
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-minimal_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-venv_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-stdlib_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-dev_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/mime-support_3.66_all.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/binfmt-support_2.2.1-2_amd64.deb
/kaggle/input/konwinski-prize/kprize

In [2]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
import os
import shutil

import pandas as pd
import polars as pl

import kaggle_evaluation.konwinski_prize_inference_server


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/konwinski-prize/data.a_zip
/kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.1-py3-none-any.whl
/kaggle/input/konwinski-prize/kprize_setup/uv/uv
/kaggle/input/konwinski-prize/kprize_setup/uv/uvx
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-minimal_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-venv_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-stdlib_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-dev_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/mime-support_3.66_all.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/binfmt-support_2.2.1-2_amd64.deb
/kaggle/input/konwinski-prize/kprize

In [3]:
import os
import io
import time
import shutil
import threading
import sys
import torch
import zipfile
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import kaggle_evaluation.konwinski_prize_inference_server

In [4]:
# Path to ZIP file and extraction directory
zip_path = "/kaggle/input/konwinski-prize/data.a_zip"
extract_dir = "/kaggle/working/unzipped_data/"
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Path to dataset (Parquet format)
parquet_file = os.path.join(extract_dir, "data", "data.parquet")

# Load data into Pandas DataFrame
df = pd.read_parquet(parquet_file)
print(df.head())  # Display first few rows

                instance_id                repo  \
0  pylint-dev__astroid-2496  pylint-dev/astroid   
1  pylint-dev__astroid-2468  pylint-dev/astroid   
2    astropy__astropy-17048     astropy/astropy   
3    astropy__astropy-16898     astropy/astropy   
4    astropy__astropy-16830     astropy/astropy   

                                   problem_statement  \
0  TypeError: unsupported format string passed to...   
1  Pylint checks against incorrect type with prop...   
2  QTable cannot take `dimensionless_unscaled` wh...   
3  BUG: tables do not deal well with zero-sized s...   
4  KeyError: 'version_1_3_or_later' when parsing ...   

                                               patch  \
0  diff --git a/ChangeLog b/ChangeLog\nindex 4560...   
1  diff --git a/ChangeLog b/ChangeLog\nindex fdbb...   
2  diff --git a/astropy/table/table.py b/astropy/...   
3  diff --git a/astropy/io/registry/core.py b/ast...   
4  diff --git a/astropy/io/votable/tree.py b/astr...   

                   

In [5]:
# Global variables for first-time prediction
first_prediction = True
model = None
tokenizer = None

# Model path inside Kaggle dataset
MODEL_PATH = "/kaggle/input/starcoder2/transformers/3b/1"

def load_model():
    """Loads the model in the background after the first prediction."""
    global model, tokenizer
    sys.stderr.write("Loading model in background...\n")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16, device_map="auto")

    sys.stderr.write("Model loaded successfully!\n")

In [6]:
def generate_patch(problem_text, repo):
    """Generates patch using the LLM model."""
    prompt = f"""
    [Repo]: {repo}
    [Problem]: {problem_text}
    [Patch]: 
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024, padding=True).to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=512,  # Reduce tokens for faster inference
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [7]:
# Paths for extracted repositories
REPO_PATH = "/tmp/repo"
ARCHIVE_PATH = "/tmp/repo_archive.tar"

log_file = "/kaggle/working/inference_logs.txt"

def log_inference_time(instance_id, time_taken):
    """Logs inference time for each instance."""
    with open(log_file, "a") as f:
        f.write(f"Instance {instance_id}: {time_taken:.2f} seconds\n")
    print(f"Instance {instance_id}: {time_taken:.2f} seconds")

def predict(problem_statement: str, repo_archive: io.BytesIO, pip_packages_archive: io.BytesIO, env_setup_cmds_templates: list[str]) -> str:
    """Handles predictions and logs inference time."""
    global first_prediction, model, tokenizer

    # First prediction - triggers background model loading
    if first_prediction:
        first_prediction = False
        threading.Thread(target=load_model, daemon=True).start()
        return ""

    # Ensure model is loaded
    if model is None or tokenizer is None:
        return "Error: Model not loaded yet."

    # Track inference start time
    start_time = time.time()

    # Extract repository
    if os.path.exists(REPO_PATH):
        shutil.rmtree(REPO_PATH)

    with open(ARCHIVE_PATH, "wb") as f:
        f.write(repo_archive.read())

    shutil.unpack_archive(ARCHIVE_PATH, extract_dir=REPO_PATH)
    os.remove(ARCHIVE_PATH)

    # Generate patch
    result = generate_patch(problem_statement, REPO_PATH)

    # Log inference time
    end_time = time.time()
    inference_time = end_time - start_time
    log_inference_time(problem_statement[:30], inference_time)  # Log first 30 chars of problem

    # Print problem statement and generated solution
    print(f"\n Problem Statement:\n{problem_statement}\n")
    print(f" Generated Patch:\n{result}\n")

    return result

In [8]:

# Define a named function for receiving instance count
def get_number_of_instances(num_instances: int) -> None:
    """Receives the number of instances to be processed. Required by the server."""
    global instance_count
    instance_count = num_instances

In [9]:
# Initialize inference server
inference_server = kaggle_evaluation.konwinski_prize_inference_server.KPrizeInferenceServer(
    get_number_of_instances,
    predict
)

# Run inference
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            "/kaggle/input/konwinski-prize/",  
            "/kaggle/tmp/konwinski-prize/",  
        ),
        use_concurrency=True
    )

Existing uv installation found. Skipping uv installation.
Installing Python 3.11...


Loading model in background...
